# Introduction
This script is served for preference datasete generation for RLHF algorithm, including code summary task and code generation task. In this code, Gemini Pro model out is treated as the chosen one and Phi-2 model output is treated as the rejected one. Before start, please download the [leetcode dataset](https://huggingface.co/datasets/RayBernard/leetcode/tree/main) and store it in a "leetcode" folder.


Reference: [tutorial for Genemi API.ipynb](https://github.com/DylanJamesZapzalka/eecs-545-project/blob/main/tutorial%20for%20Genemi%20API.ipynb).

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

## Gemini

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 2.5 MB/s eta 0:00:00


In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

# Used to securely store your API key
from google.colab import userdata

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


Setup your API key
Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

[Get an API key](https://aistudio.google.com/app/apikey)

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name GOOGLE_API_KEY.

Once you have the API key, pass it to the SDK. You can do this in two ways:

Put the key in the GOOGLE_API_KEY environment variable (the SDK will automatically pick it up from there).
Pass the key to genai.configure(api_key=...)

In [ ]:
# You should configure your API_KEY see
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')

### Code summary

In [ ]:
data_file =  '/content/leetcode/'
dataset = load_dataset(data_file)
Python_problems = dataset["train"]['output']  ## Python code
preferred_summary = []
Python_problems[0]

'```python\ndef twoSum(nums, target):\n    map = {}\n    for i, num in enumerate(nums):\n        complement = target - num\n        if complement in map:\n            return [map[complement], i]\n        map[num] = i\n    return []\n```\n\n'

In [ ]:
import time
start_index = len(preferred_summary)
for i in range(start_index, len(Python_problems)):
  prompt = "Given the following Python code, provide a summary of its functionality:\n" + Python_problems[i]
  try: # prevent from stream limitation
    response = model.generate_content(prompt)
  except:
    time.sleep(5)
    response = model.generate_content(prompt)

  try: # prevent from unsuccesful completion
    code_summary = response.text
    preferred_summary.append(code_summary)
  except:
    preferred_summary.append("")

  print(i)


In [ ]:
len(preferred_summary)

2359

### Code generation

In [ ]:
data_file =  '/content/leetcode/'
dataset = load_dataset(data_file)
Python_problems = dataset["train"]['input']  ## Python problem description
preferred_solution = []
Python_problems[0]

In [ ]:
import time
start_index = len(preferred_solution)
for i in range(start_index, len(Python_problems)):
  prompt = "### Indtruction:\n Write a Python function that solves the following problem. Your function should take appropriate input (if any) and return the expected output. Feel free to provide any additional context or constraints necessary for solving the problem.\n" + "### Question \n" + Python_problems[i]
  try: # prevent from stream limitation
    response = model.generate_content(prompt)
  except:
    time.sleep(5)
    response = model.generate_content(prompt)

  try: # prevent from unsuccesful completion
    code_summary = response.text
    preferred_solution.append(code_summary)
  except:
    preferred_solution.append("")

  print(i)


## Phi-2

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
# torch.cuda.empty_cache()
torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Code summary

In [ ]:
data_file =  '/content/leetcode/'
dataset = load_dataset(data_file)
Python_problems = dataset["train"]['output']  ## Python code
to_markdowm(Python_problems[0])

In [ ]:
import time
preferred_summary = []
start_index = len(preferred_summary)
for i in range(start_index, len(Python_problems)):
  prompt = "Given the following Python code, provide a summary of its functionality:\n" + Python_problems[i]
  inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to('cuda')

  outputs = model.generate(**inputs, max_length=len(prompt)+300, pad_token_id=tokenizer.eos_token_id)
  text = tokenizer.batch_decode(outputs)[0]

  rejected_summary.append(text)

  print(i)


### Code Generation

In [ ]:
data_file =  '/content/leetcode/'
dataset = load_dataset(data_file)
Python_problems = dataset["train"]['input']  ## Python problem description
to_markdowm(Python_problems[0])

In [ ]:
rejected_solution = []
for i in range(len(Python_problems)):
  prompt = "Write a Python function that solves the following problem. Your function should take appropriate input (if any) and return the expected output. Feel free to provide any additional context or constraints necessary for solving the problem.\n" + "### Question \n" + Python_problems[i]
  inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)

  outputs = model.generate(**inputs, max_length=500, pad_token_id=tokenizer.eos_token_id)

  text = tokenizer.batch_decode(outputs)[0]

  rejected_solution.append(text)

  print(i)
